In [1]:
import random
from Crypto.Util.number import long_to_bytes, bytes_to_long, inverse, GCD

# Prerequisites

- miller-rabin
- modular arithmeric
- patience to understand the algorithm

In [19]:
def miller_rabin_test_fixed(n, a_list):
    '''returns Composite or Probably prime'''
    
    #check parity
    if not n & 1:
        return 'Composite'
    
    r = 0
    s = n-1
    #write n-1 = 2^r*s
    while not s & 1:
        s = s>>1
        r+=1
    assert(pow(2, r)* s == n-1)
    
    for a in a_list:
        x = pow(a, s, n)
        if x == 1 or x == n-1:
            continue #search for another witness
        for _ in range(r):
            x = pow(x, 2, n)
            if x == n-1:
                break 
        else:
             #if it doesnt break, neither condition is satisfied =>  this executes => we found a composite
            return "Composite"
    return "Probably prime"
    

# Theory

- https://core.ac.uk/download/pdf/81930829.pdf
- https://www.jointmathematicsmeetings.org/mcom/1995-64-209/S0025-5718-1995-1260124-2/S0025-5718-1995-1260124-2.pdf

Let $n = p_1p_2..p_h$
Define: 
- $k_i  = \cfrac {p_i - 1} {p_1 - 1}$
- $m_i = \cfrac {\Pi_{j \neq i} {p_j - 1}} {p_1 - 1} \forall 1 \leq i \leq h$

If $k_i, m_i \in \mathbb{Z} \ \forall 2 \leq i \leq h => n$ is a Carmichael number

(The law of quadratic reciprocity) If $p$ and $q$ are distinct odd primes, we have
$\left(\dfrac{q}{p}\right)\left(\dfrac{p}{q}\right)=(-1)^{\frac{p-1}{2} \frac{q-1}{2}}$


Additional proprieties:
- if $\gcd(a, n) = 1$ and $\left(\dfrac a {p_i} \right)= -1$ for all $1 \leq i \leq h$ then $a$ will be a Miller Rabin non-witness with respect to $n$ 
- for any prime $p$,  $\left(\dfrac a {p}\right)$ can be determined from $\left(\dfrac p a\right)$ and the values of $a \bmod 4$ and $p \bmod 4$ => for each a we can compute the set $S_a$ of possible non-residues mod $4a$ of potential primes $p$
    $$S_a \text{ satisfying } \left(\dfrac a {p}\right) = -1 \iff p \bmod 4a \in S_a$$
    
Starting with some $p_1$ we can determine the other $p_i$ of the form $p_i = k_i(p_1 - 1)+ 1$ for all $1 \leq i \leq h => $ $$k_i(p_1 - 1) + 1 \bmod 4a \in S_a \ \forall 1\leq i \leq h$$

If the coeff $k_i$ are coprime to $a$ then the condition becomes:$$p_1 \bmod 4a∈⋂^h_{i=1}k^{−1}_i(S_a+k_i−1)$$ where $k^{−1}_i(S_a+k_i−1)$ is the set $\{k^{−1}_i(s+k_i−1) | s \in S_a\}$ => We have a set of conditions for the values of $p_1$ for each $a$

So for each value of $a$ we have a few candidates in our subset from $S_a$. We select one of these candidates $z_a$ for each $a \in A$ and using the CRT we can combine these conditions into a single one $p_1 \bmod \text{lcm}(4, a_1, .. a_h)$ 

The $k_i$ are a set of primes usually smaller than $\max(A)$ such that $k_i^{-1}$ exists mod $4a$

**Algorithm**:
- Generate the set $S_a$ of residues modulo $4a$ of primes $p$ s.t $\left(\dfrac a {p}\right) = -1$ for each base $a$
- Select $k_i$ coprime to all $a$
- Find the subsets that meet the condition 
    - $p_1 \bmod 4a∈⋂^h_{i=1}k^{−1}_i(S_a+k_i−1)$
- Choose 1 residue / set => $z_a$ (arbitrary but not all combinations will lead to a solution)
- Verify additional conditions:
    - for $h = 3 $
        - $p_1 \equiv k_3^{-1} \bmod k_2$
        - $p_1 \equiv k_2^{-1} \bmod k_3$
- Use CRT with all the conditions 
    - $p_1 = z \bmod \text{lcm}(4, a_1, ... a_h, k_2, ... k_h)$
- Compute $p_i = k_i(p_1 - 1) + 1$ such that all $p_i$ are prime.

# Code

## Example in the Prime and Prejudice paper

In [5]:
def generate_S_a(a_list, p_bound):
    # generate residues of primes mod 4 * a with (a / p ) = -1 for each base a
    S = {a: set() for a in a_list}
    for a in a_list:
        p = 3
        while p < p_bound:
            if legendre_symbol(a, p) == -1:
                S[a].add(p % (4 * a))
            p = next_prime(p)
    return S

In [6]:
#a_list = [2,3,5,7,11,13,17,19,23,29]
a_list = [2, 3, 5, 7]
S = generate_S_a(a_list, 10000)
#S

In [8]:
def get_k(h, a_list, prime_bound):
    #there are different ways to generate k.
    #We will just take random primes up to a bound that are not in the base list
    k_list = [1]
    for i in range(1, h):
        while True:
            k = random_prime(prime_bound, lbound=2)
            if k not in a_list and k not in k_list:
                k_list.append(k)
                break
    return k_list

In [63]:
h = 3
k_list = get_k(h, a_list, 83)
#k_list = [1, 41, 101]
k_list

[1, 13, 43]

In [64]:
def find_subsets_S(S, a_list,  k_list):
    S_sub = {a: set() for a in a_list}
    
    for a in a_list:
        temp_sets = []
        
        for k in k_list: #create set subset for each k
            temp_set = set()
            inv_k = inverse_mod(k, 4 * a)
            for elem in S[a]: 
                new_el = (inv_k * (elem + k - 1)) % (4 * a)
                temp_set.add(new_el)
            temp_sets.append(temp_set)
        
        S_sub[a] = set.intersection(*temp_sets)
    return S_sub
        

In [65]:
#find sets that are not empty
while True:
    k_list = get_k(h, a_list, 83)
    S_sub = find_subsets_S(S, a_list, k_list)
    for s in S_sub.values(): #check if sets are empty
        if len(s)==0:
            break
    else:
        break #exit the while True if all sets are not empty

In [66]:
S_sub

{2: {5}, 3: {5, 7}, 5: {3, 13}, 7: {13, 15, 17}}

In [67]:
def choose_z(S_sub, a_list):
    # Choose randomly but not all combinations will lead to a solution
    zs = {a : random.choice(tuple(S_sub[a])) for a in a_list}
    return zs

In [68]:
zs = choose_z(S_sub, a_list)
#zs = {2: 3, 3: 7, 5: 3, 7: 15, 11: 23, 13: 47, 17: 31, 19: 47, 23: 47, 29:55}
zs

{2: 5, 3: 5, 5: 13, 7: 13}

In [69]:
print(k_list)
inverse_mod(-k_list[1], k_list[2]), inverse_mod(-k_list[2], k_list[1])

[1, 13, 43]


(33, 3)

In [70]:
def create_and_solve_conditions(a_list, zs, k_list):
    # h = 3 example
    
    #create crt conditions
    #will do with the polynomial later
    crt_a_list = [zs[a] for a in a_list] + [inverse_mod(-k_list[1], k_list[2]), inverse_mod(-k_list[2], k_list[1])]
    crt_m_list = [4*a for a in a_list] + [k_list[2], k_list[1]]
    
    #solve
    m = product(crt_m_list)
    x = crt(crt_a_list, crt_m_list)
    return x, m
    

In [71]:
x, m = create_and_solve_conditions(a_list, zs, k_list)

In [72]:
x, m

(300173, 30051840)

In [90]:
def get_pseudoprime(x, m, k_list, starting_bitsize):
    #search for prime p1 = k * m + x
    if starting_bitsize >  int(m).bit_length():
        i = 2**(starting_bitsize - int(m).bit_length())
    else:
        i = 1
    
    p1 = i * m + x
    print(int(p1).bit_length())
    while True:
        if i % 1000 == 0:
            print(i)
        if is_prime(p1):
            #start constructing the list
            p_list = [p1]
            for k in k_list[1:]:
                p_new = k * (p1 - 1) + 1
                if is_prime(p_new):
                    p_list.append(p_new)
                else: #if one of the generated numbers is not prime break out the loop. The for-else doesnt get executed
                    break
            else: #if we find all primes break out the while True
                return product(p_list), p_list
        
        p1 += m        

In [91]:
int(142445387161415482404826365418175962266689133006163).bit_length()

167

In [95]:
p, p_list = get_pseudoprime(x, m, k_list, starting_bitsize = 167)


167


In [96]:
miller_rabin_test_fixed(p, a_list)

'Probably prime'

In [88]:
p_list

[330870413, 330870413, 4301315357, 14227427717]

## Class time (same as above but put in a class)

In [11]:
class ArnaultPseudoprime:
    def __init__(self, a_list, gen_S_prime_bound = 1000, zs_iter_bound = 100, k_bound = 300):
        self.gen_S_prime_bound = gen_S_prime_bound
        self.zs_iter_bound = zs_iter_bound
        self.k_bound = k_bound
        self.a_list = a_list
        
        
    def find_pseudoprime(self, p1_starting_bitsize = 1):
        '''Main function to find pseudoprime'''
        #find sets that are not empty
        S = self.generate_S_a(self.a_list, self.gen_S_prime_bound)
        h = 3
        i = 0
        print("starting searching for a good condition set")
        while True:
            if i % self.zs_iter_bound == 0: #reset k after zs_iter_bound tries
                while True:#find sets that are not empty
                    k_list = self.get_k(h, self.a_list, self.k_bound)
                    S_sub = self.find_subsets_S(S, self.a_list, k_list)
                    
                    for s in S_sub.values(): #check if sets are empty
                        if len(s)==0: #if it's empty break and the else doesn't execute
                            break
                    else:
                        #print(i)
                        break #exit the while True if all sets are not empty
                        
            #try to solve congruences
            try:
                zs = self.choose_z(S_sub, self.a_list) #chose random zs
                x, m = self.create_and_solve_conditions(self.a_list, zs, k_list) 
                break
            except Exception as e:
                i+=1
                continue
                
        #get pseudoprime
        print("starting searching for a pseudoprime")
        p, p_list = self.get_pseudoprime(x, m, k_list, starting_bitsize = p1_starting_bitsize)
        return p, p_list

        
    def generate_S_a(self, a_list, p_bound):
        '''generate residues of primes mod 4 * a with (a / p ) = -1 for each base a'''
        S = {a: set() for a in a_list}
        for a in a_list:
            p = 3
            while p < p_bound:
                if legendre_symbol(a, p) == -1:
                    S[a].add(p % (4 * a))
                p = next_prime(p)
        return S


    def get_k(self, h, a_list, prime_bound):
        '''
        there are different ways to generate k.
        We will just take random primes up to a bound that are not in the base list
        '''
        k_list = [1]
        for i in range(1, h):
            while True:
                k = random_prime(prime_bound, lbound=2)
                if k not in a_list and k not in k_list:
                    k_list.append(k)
                    break
        return k_list



    def find_subsets_S(self, S, a_list,  k_list):
        S_sub = {a: set() for a in a_list}

        for a in a_list:
            temp_sets = []

            for k in k_list: #create set subset for each k
                temp_set = set()
                inv_k = inverse_mod(k, 4 * a)
                for elem in S[a]: 
                    new_el = (inv_k * (elem + k - 1)) % (4 * a)
                    temp_set.add(new_el)
                temp_sets.append(temp_set)

            S_sub[a] = set.intersection(*temp_sets)
        return S_sub
        

    def choose_z(self, S_sub, a_list):
        '''Choose randomly but not all combinations will lead to a solution'''
        zs = {a : random.choice(tuple(S_sub[a])) for a in a_list}
        return zs

    
    def create_and_solve_conditions(self, a_list, zs, k_list):
        '''
        h = 3 example
        create crt conditions
        will do with the polynomial later
        '''
        crt_a_list = [zs[a] for a in a_list] + [inverse_mod(-k_list[1], k_list[2]), inverse_mod(-k_list[2], k_list[1])]
        crt_m_list = [4*a for a in a_list] + [k_list[2], k_list[1]]

        #solve
        m = product(crt_m_list)
        x = crt(crt_a_list, crt_m_list)
        return x, m

    def get_pseudoprime(self, x, m, k_list, starting_bitsize):
        '''search for prime p1 = k * m + x'''
        if starting_bitsize >  int(m).bit_length():
            i = 2**(starting_bitsize - int(m).bit_length())
        else:
            i = 1

        p1 = i * m + x
        print(int(p1).bit_length())
        while True:
            if i % 1000 == 0:
                print(i)
            if is_prime(p1):
                #start constructing the list
                p_list = [p1]
                for k in k_list[1:]:
                    p_new = k * (p1 - 1) + 1
                    if is_prime(p_new):
                        p_list.append(p_new)
                    else: #if one of the generated numbers is not prime break out the loop. The for-else doesnt get executed
                        break
                else: #if we find all primes break out the while True
                    return product(p_list), p_list
        
            p1 += m        

In [12]:
a_list =[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61]
arn = ArnaultPseudoprime(a_list, gen_S_prime_bound = 1000, zs_iter_bound = 100, k_bound = 300)

In [17]:
p, p_list = arn.find_pseudoprime(p1_starting_bitsize=200)

starting searching for a good condition set
starting searching for a pseudoprime
200


In [20]:
miller_rabin_test_fixed(p, a_list)

'Probably prime'